In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksstorage222.dfs.core.windows.net/customers")
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("domains", split(col('email'), '@')[1])
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("count")).sort(desc("count")).display()


In [0]:
df_gmail = df.filter(col("domains") == "gmail.com")
df_gmail.display()

df_yahoo = df.filter(col("domains") == "yahoo.com")
df_yahoo.display()

df_hotmail = df.filter(col("domains") == "hotmail.com")
df_hotmail.display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@databricksstorage222.dfs.core.windows.net/customers")

In [0]:
df = spark.read.format("delta").load("abfss://silver@databricksstorage222.dfs.core.windows.net/customers")
df.display()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@databricksstorage222.dfs.core.windows.net/customers'

In [0]:
%sql
select *
from databricks_catalog.silver.customers_silver